In [1]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from lstm_autoencoder_source import CNN_Autoencoder
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import cv2
import pickle

In [2]:
kind = 'lunge'
with open(f"{kind}_dataloader.pkl", "rb") as f:
    saved_data = pickle.load(f)

loaded_dataset = saved_data["dataset"]
dataloader = DataLoader(
    loaded_dataset,
    batch_size=saved_data["batch_size"],
    shuffle=saved_data["shuffle"],
)

In [3]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
model = CNN_Autoencoder().to(device)
def saveModel():
    torch.save(model.state_dict(), f'model/{kind}_train.pt')
criterion = nn.BCELoss()
optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer,T_max= 25, eta_min=1e-3)
# 학습 루프
epochs = 10
min_loss = 1e+10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for framex,framey in dataloader:
        input = framex.to(device)        
        optimizer.zero_grad()

        # 순전파
        output = model(input)

        # 손실 계산
        loss = criterion(output,framey.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    mean_loss =running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(dataloader)}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()
    

Epoch [1/10], Loss: 0.6450609139033726
model_saved
Epoch [2/10], Loss: 0.6366143992968968
model_saved
Epoch [3/10], Loss: 0.6259746551513672
model_saved
Epoch [4/10], Loss: 0.5247619322368077
model_saved
Epoch [5/10], Loss: 0.22056557663849422
model_saved
Epoch [6/10], Loss: 0.09695594970669065
model_saved
Epoch [7/10], Loss: 0.06168675582323756
model_saved
Epoch [8/10], Loss: 0.04572027336273875
model_saved
Epoch [9/10], Loss: 0.03625815042427608
model_saved
Epoch [10/10], Loss: 0.03114562907389232
model_saved
